### Set Up Birthright

In [1]:
import pandas as pd
import numpy as np



cols= ['Provence', 'Regent', 'Type', 'Level']
Holdings = pd.DataFrame(columns=cols)    

def get_my_index(df, temp):
    if len(temp) == 1:
        index = temp[0]
    else:
        try:
            index = max(df.index.tolist()) + 1
        except:
            print('new')
            index=0
    return index

def add_holding(Provence, Regent, Type='Law', Level=1, df=Holdings):
    '''
     Provence: match to provence
     Regent: match to regent
     Type: 'Law', 'Guild', 'Temple', 'Source' 
     Level: 1 to Population (or Magic for source)
     
     temp = Holdings[Holdings['Regent']==Regent]
temp = temp[temp['Type']==Type]
temp.index[temp['Provence'] == 'Bogsend' ].tolist()
    '''

    temp = df[df['Provence']==Provence]
    temp = temp[temp['Regent']==Regent]
    temp = temp.index[temp['Type']==Type].tolist()
    index = get_my_index(df, temp)
    
    df.loc[index] = [Provence, Regent, Type, Level]
    df['Level'] = df['Level'].astype(int)
    return df

def remove_holding(Provence, Regent, Type, df=Holdings):
    '''
    Remove all rows where Regent, Provence, Type are
    equakl to those set.
    '''
    temp = df[df['Provence']==Provence] # just the provence in question
    df = df[df['Provence'] != Provence] # all others are safe
     # add back all other regents ion that provence
    df = pd.concat(df, temp[temp['Regent'] != Regent]) 
    # isolate regent
    temp = temp[temp['Regent'] == Regent]
    # add back all other types
    df = pd.concat(df, temp[temp['Regent'] != Type])
    
    #done
    return df


cols = ['Provence', 'Domain', 'Regent', 'Terrain', 'Loyalty', 'Taxation',
        'Population', 'Magic', 'Castle', 'Capital', 'Position', 'Troops']
Provences = pd.DataFrame(columns=cols)

def add_provence(Provence, Domain, Regent, x, y, df=Provences
                 , Population=0, Magic=1, Law=None
                 , Capital=False, Terrain='Plains', Loyalty='Average', Taxation='Moderate'
                 , Castle=0, Holdings=Holdings, Troops=0):
    '''
    Provence: pkey, Name
    Domain: Name 
    Regent: Name, foreign key on Regents 
    Terrain: 'Desert', 'Tundra', 'Forest', 'Mountain', 'Glacier', 'Hills', 
            'Plains', 'Farmland', 'Steppes', 'Swamp', 'Marsh' 
    Taxation: 'Light', 'Moderate', or 'Severe'
    Loyalty: 'High', 'Average', 'Poor', or 'Rebellious'
    Population: 0 thru 9
    Magic: 1 thru 9 (lowers when population increases, minimumm 1)
    Castle: 0+ (when they fortify, they get this)
    Capital: True or False if Capital of Domain
    x,y: x and y position
    Troops: String Values of type(subtype), comma seperated
    '''
    
    
    temp = df.index[df['Provence'] == 'Provence'].tolist()
    index = get_my_index(df, temp)
            
    df.loc[df.shape[0]] = [Provence, Domain, Regent, Terrain, Loyalty, Taxation,
                           Population, Magic, Castle, Capital, np.array([x, y]), Troops]
    df['Magic'] = df['Magic'].astype(int)
    df['Population'] = df['Population'].astype(int)
    df['Castle'] = df['Castle'].astype(int)
    df = df.drop_duplicates(subset='Provence', keep="last")
    
    return df
   
cols = ['Regent', 'Full Name', 'Player', 
         'Class', 'Level', 'Alignment', 'Str', 'Dex', 'Con', 'Int', 'Wis', 'Cha',
        'Insight', 'Deception', 'Persuasion',
         'Regency Points', 'Gold Bars', 'Regency Bonus', 'Attitude', 'Lieutenants']
Regents = pd.DataFrame(columns=cols)

def add_regent(Regent, Name, Player=False, df=Regents, Class='Noble', Level=2, Alignment = 'NN'
               , Str = 0, Dex = 1, Con = 0, Int = 1, Wis = 2, Cha = 3
               , Insight = 4, Deception = 5, Persuasion = 5
               , Regency_Points = 0, Gold_Bars = 0, Regency_Bonus = 1
               , Attitude = 'Normal', Lieutenants=[], Archetype=None):
    '''
    Archetype: Allows forpre-loaded skill and ability mods based on NPC statblocks
    '''
    temp = df.index[df['Regent'] == 'Regent'].tolist()
    index = get_my_index(df, temp)
    if Archetype != None:
        # set the stats based on archetype
        Class, Level, Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion = get_archetype(Archetype)

    df.loc[df.shape[0]] = [Regent, Name, Player, Class, Level, Alignment, 
                           Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion,
                           Regency_Points, Gold_Bars, Regency_Bonus, Attitude, Lieutenants]
    df = df.drop_duplicates(subset='Regent', keep='last')
    return df

def get_archetype(Archetype):
    # return  Class, Level, Str, Dex, Con, Int, Wis, Cha, Insight, Deception, Persuasion
    if Archetype == 'Noble':
        return 'Noble', 2, 0, 1, 0, 1, 2, 3, 4, 5, 5
    elif Archetype == 'Archmage':
        return 'Archmage', 18, 0, 2, 1, 5, 2, 3, 5, 3, 3
    elif Archetype == 'Assassin':
        return 'Assassin', 12, 0, 3, 2, 1, 0, 0, 1, 3, 0
    elif Archetype == 'Bandit' or Archetype == 'Bandit Captain':
        return 'Bandit Captain', 10, 2, 3, 2, 2, 0, 2, 2, 4, 2
    elif Archetype == 'Commoner':
        return 'Commoner', 1, 0, 0, 0, 0, 0, 0, 0, 0, 0
    elif Archetype == 'Druid':
        return 'Druid', 5, 0, 1, 1, 1, 2, 0, 1, 0, 0
    elif Archetype == 'Knight':
        return 'Knight', 8, 3, 0, 2, 0, 0, 2, 0, 2, 2
    elif Archetype == 'Lich':
        return 'Lich', 18, 0, 3, 3, 5, 2, 3, 9, 3, 3
    elif Archetype == 'Mage':
        return 'Mage', 9, -1, 2, 0, 3, 1, 0, 3, 0, 0
    elif Archetype == 'Hag' or Archetype == 'Green Hag':
        return 'Green Hag', 11, 4, 1, 3, 1, 2, 2, 1, 4, 2
    elif Archetype == 'Priest':
        return 'Priest', 5, 0, 0, 1, 1, 3, 1, 1, 1, 3
    
    # if none of the above, return Noble stats
    else:
        return 'Noble', 2, 0, 1, 0, 1, 2, 3, 4, 5, 5

cols = ['Provence', 'Neighbor', 'Border', 'Road', 'Caravan', 'Shipping']
Geography = pd.DataFrame(columns=cols)
    
def add_geo(Provence, Neighbor, Border=0, Road=0, Caravan=0, Shipping=0, df=Geography):
    temp = df[df['Provence'] == Provence]
    temp = temp.index[temp['Neighbor']==Neighbor].tolist()
    index = get_my_index(df, temp)
    df.loc[index] = [Provence, Neighbor, Border, Road, Caravan, Shipping]
    df.loc[index] = [Neighbor, Provence, Border, Road, Caravan, Shipping]
    return df

cols = ['Regent', 'Provence', 'Type', 'Cost', 'CR']
Troops = pd.DataFrame(columns=cols)
def add_troops(Regent, Provence, Type, Cost, CR, df=Troops):
    df[max(df.index.tolist()) + 1] = [Regent, Provence, Type, Cost, CR]
    

cols = ['Regent', 'Other', 'Diplomacy', 'Payment', 'Vassalage']
Relationships = pd.DataFrame(columns=cols)

def add_relationship(Regent, Other, Diplomacy=0, Payment=0, Vassalage=0, df=Relationships):
    '''
    Regent -> Whose Relationship
    Other -> To whom
    Diplomacy -> how much Regent Likes Other
    Payment -> How much Regent jas agreed to pay Other every season
    Vassalage -> How many of Regent's Regency Points are paid to Othervas their Liege Lord
    '''
    cols = ['Regent', 'Other', 'Diplomacy', 'Payment', 'Vassalage']
    
    
   
    temp = df[df['Regent']==Regent]
    temp = temp.index[temp['Other']==Other]
    if len(temp) > 0:  # we have a row already
        Diplomacy = Diplomacy + df.loc[temp[0]]['Diplomacy']
        Payment = Payment + df.loc[temp[0]]['Payment']
        Vassalage = Vassalage + df.loc[temp[0]]['Vassalage']
    index = get_my_index(df, temp)
    if Diplomacy + Payment + Vassalage == 0:
        return df
    else:
        df.loc[index] = [Regent, Other, Diplomacy, Payment, Vassalage]
        return df

In [2]:
domain='Coeranys'
Regents = add_regent('EC', 'Eluvie Cariele', Player=True, Class='Fighter', Alignment='CG', Regency_Bonus=2
                     , Attitude='Peaceful', Lieutenants=['Aedric Bherenstae'])
Regents = add_regent('LPA', 'Life and Protection of Avanalae (Medhlorie Haensen)', Archetype='Priest', Regency_Bonus=3)
Regents = add_regent('GH', 'Ghorien Hiriele (Highland/Overland Traders)')
Regents = add_regent('Sw2', 'Second Swamp Mage', Archetype='Hag')
Regents = add_regent('HA', "Haelyn's Aegis (Anita Maricoere)", Archetype='Priest')
Regents = add_regent('DW', "Diirk Watersold (Royal Guild of Baruk-Azhir)")
Regents = add_regent('TBM', "Three Brother Mages", Archetype='Mage')
Regents = add_regent('EL', "Elamien Lamier (Points East Trading Guild)")

Relationships = add_relationship('LPA', 'EC', Diplomacy=3, Vassalage=1)

p = 'Bogsend'
Provences = add_provence(p, domain, 'EC', 1543, 1985, Population=3, Magic=2, Terrain='Swamp')
Holdings = add_holding(p, 'EC', 'Law', 1)
Holdings = add_holding(p, 'LPA', 'Temple', 1)
Holdings = add_holding(p, 'GH', 'Guild', 3)
Holdings = add_holding(p, 'Sw2', 'Source', 2)


p = 'Caudraight'
Provences = add_provence(p, domain, 'EC', 1556, 1914, Population=4, Magic=1)
Holdings = add_holding(p, 'EC', 'Law', 2)
Holdings = add_holding(p, 'LPA', 'Temple', 2)
Holdings = add_holding(p, 'A', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'GH', 'Guild', 2)

p = 'Deepshadow'
Provences = add_provence(p, domain, 'EC', 1562, 1817, Population=3, Magic=2)
Holdings = add_holding(p, 'EC', 'Law', 2)
Holdings = add_holding(p, 'LPA', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 3)
Holdings = add_holding(p, 'TBM', 'Source', 3)

p = 'Duornil'
Provences = add_provence(p, domain, 'EC', 1606, 1837, Population=2, Magic=3)
Holdings = add_holding(p, 'EC', 'Law', 2)
Holdings = add_holding(p, 'LPA', 'Temple', 1)
Holdings = add_holding(p, 'DW', 'Guild', 3)
Holdings = add_holding(p, 'TBM', 'Source', 2)

p = 'Mistil'
Provences = add_provence(p, domain, 'EC', 1662, 1966, Population=2, Magic=3)
Holdings = add_holding(p, 'EC', 'Law', 1)
Holdings = add_holding(p, 'LPA', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'TBM', 'Source', 3)

p = 'Ranien'
Provences = add_provence(p, domain, 'EC', 1610, 1921, Population=2, Magic=3)
Holdings = add_holding(p, 'EC', 'Law', 1)
Holdings = add_holding(p, 'LPA', 'Temple', 1)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'TBM', 'Source', 3)

p = 'Ruorven'
Provences = add_provence(p, domain, 'EC', 1610, 2043, Population=4, Magic=1, Terrain='Swamp', Capital=True)
Holdings = add_holding(p, 'EC', 'Law', 3)
Holdings = add_holding(p, 'LPA', 'Temple', 3)
Holdings = add_holding(p, 'HA', 'Temple', 1)
Holdings = add_holding(p, 'EL', 'Guild', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)

Geography=add_geo('Deepshadow', 'Duornil', Border=1)
Geography=add_geo('Caudraight', 'Duornil', Border=1)
Geography=add_geo('Deepshadow', 'Caudraight', Border=1)
Geography=add_geo('Caudraight', 'Ranien', Border=1)
Geography=add_geo('Caudraight', 'Bogsend', Border=1)
Geography=add_geo('Mistil', 'Ranien', Border=1)
Geography=add_geo('Mistil', 'Ruorven', Border=1)
Geography=add_geo('Bogsend', 'Ruorven', Border=1)
Geography=add_geo('Bogsend', 'Ranien', Border=1)
Geography=add_geo('Ranien', 'Ruorven', Border=1)
Geography=add_geo('Ranien', 'Duornil', Border=1)

domain='Osoerde'

Geography=add_geo('Brothendar', 'Algael', Border=1)
Geography=add_geo('Brothendar', 'Ghalliere', Border=1)
Geography=add_geo('Brothendar', 'Gulfport', Border=1)
Geography=add_geo('Moergen', 'Ghalliere', Border=1)
Geography=add_geo('Moergen', 'Gulfport', Border=1)
Geography=add_geo('Moriel', 'Ghalliere', Border=1)
Geography=add_geo('Moriel', 'Gulfport', Border=1)
Geography=add_geo('Moriel', 'Brothendar', Border=1)
Geography=add_geo('Moriel', 'Moergen', Border=1)
Geography=add_geo('Spiritsend', 'Algael', Border=1)
Geography=add_geo('Spiritsend', 'Gulfport', Border=1)
Geography=add_geo('Spiritsend', 'Brothendar', Border=1)
Geography=add_geo('Spiritsend', 'Moriel', Border=1)
Geography=add_geo('Sunken Lands', 'Gulfport', Border=1)
Geography=add_geo('Sunken Lands', 'Moergen', Border=1)

Geography=add_geo('Sunken Lands', 'Bogsend', Border=1)
Geography=add_geo('Bogsend', 'Moergen', Border=1)
Geography=add_geo('Sunken Lands', 'Ruorven', Border=1)
Geography=add_geo('Ruorven', 'Moergen', Border=1)

Regents = add_regent('JR', 'Jaison Raenech', Class='Fighter', Level=7, Alignment='LE', Regency_Bonus=3
                     , Attitude='Aggresive', Lieutenants=['Terence Gryphon'])
Regents = add_regent('WM', 'Willaim Moergen', Class='Fighter', Level=5, Alignment='CG', Regency_Bonus=3)
Regents = add_regent('IHH', 'Impregnable Heart of Haelyn (Hubaere Armeindin)', Archetype='Priest')
Regents = add_regent('Sw', 'Swamp Mage', Archetype='Hag')
Regents = add_regent('CSH', "Church of Storm's End", Archetype='Priest')
Regents = add_regent('TCV', 'One True Church of Vosgaard', Archetype='Priest')

Relationships = add_relationship('JR', 'WM', Diplomacy=-2)
Relationships = add_relationship('WM', 'JR', Diplomacy=-3)

p = 'Algael'
Provences = add_provence(p, domain, 'JR', 1355, 2226, Population=2, Magic=3, Terrain='Swamp')
Holdings = add_holding(p, 'JR', 'Law', 2)
Holdings = add_holding(p, 'IHH', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'Sw', 'Source', 3)

p = 'Brothendar'
Provences = add_provence(p, domain, 'JR', 1377, 2177, Population=3, Magic=2)
Holdings = add_holding(p, 'JR', 'Law', 3)
Holdings = add_holding(p, 'CSH', 'Temple', 3)
Holdings = add_holding(p, 'GH', 'Guild', 3)
Holdings = add_holding(p, 'Sw', 'Source', 2)

p = 'Ghalliere'
Provences = add_provence(p, domain, 'JR', 1394, 2110, Population=2, Magic=3)
Holdings = add_holding(p, 'JR', 'Law', 1)
Holdings = add_holding(p, 'IHH', 'Temple', 2)
Holdings = add_holding(p, 'GH', 'Guild', 2)
Holdings = add_holding(p, 'Sw2', 'Source', 3)

p = 'Gulfport'
Provences = add_provence(p, domain, 'JR', 1515, 2154, Population=3, Magic=2)
Holdings = add_holding(p, 'JR', 'Law', 3)
Holdings = add_holding(p, 'CSH', 'Temple', 2)
Holdings = add_holding(p, 'Sw2', 'Source', 2)

p = 'Moergen'
Provences = add_provence(p, domain, 'JR', 1471, 2067, Population=2, Magic=3)
Holdings = add_holding(p, 'WM', 'Law', 2)
Holdings = add_holding(p, 'IHH', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'Sw2', 'Source', 3)

p = 'Moriel'
Provences = add_provence(p, domain, 'JR', 1448, 2138, Population=4, Magic=1, Capital=True)
Holdings = add_holding(p, 'WM', 'Law', 1)
Holdings = add_holding(p, 'JR', 'Law', 3)
Holdings = add_holding(p, 'CSH', 'Temple', 2)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'Sw2', 'Source', 1)

p = 'Spiritsend'
Provences = add_provence(p, domain, 'JR', 1466, 2231, Population=2, Magic=6, Terrain='Swamp')
Holdings = add_holding(p, 'JR', 'Law', 1)
Holdings = add_holding(p, 'DW', 'Guild', 2)
Holdings = add_holding(p, 'Sw', 'Source', 6)

p = 'Sunken Lands'
Provences = add_provence(p, domain, 'JR',  1541, 2085, Population=2, Magic=6, Terrain='Swamp')
Holdings = add_holding(p, 'WM', 'Law', 2)
Holdings = add_holding(p, 'CSH', 'Temple', 2)
Holdings = add_holding(p, 'Sw2', 'Source', 6)

Birthright = {}
Birthright['Provences'] = Provences
Birthright['Holdings'] = Holdings
Birthright['Regents'] = Regents
Birthright['Geography'] = Geography
Birthright['Relationships'] = Relationships
Birthright['Troops'] = Troops


new
new
new
new
new


In [3]:
import pickle
with open('Birthright.pickle', 'wb') as handle:
    pickle.dump(Birthright, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
Regents

,Regent,Full Name,Player,Class,Level,Alignment,Str,Dex,Con,Int,Wis,Cha,Insight,Deception,Persuasion,Regency Points,Gold Bars,Regency Bonus,Attitude,Lieutenants
0,EC,Eluvie Cariele,True,Fighter,2,CG,0,1,0,1,2,3,4,5,5,0,0,2,Peaceful,[Aedric Bherenstae]
1,LPA,Life and Protection of Avanalae (Medhlorie Hae...,False,Priest,5,NN,0,0,1,1,3,1,1,1,3,0,0,3,Normal,[]
2,GH,Ghorien Hiriele (Highland/Overland Traders),False,Noble,2,NN,0,1,0,1,2,3,4,5,5,0,0,1,Normal,[]
3,Sw2,Second Swamp Mage,False,Green Hag,11,NN,4,1,3,1,2,2,1,4,2,0,0,1,Normal,[]
4,HA,Haelyn's Aegis (Anita Maricoere),False,Priest,5,NN,0,0,1,1,3,1,1,1,3,0,0,1,Normal,[]
5,DW,Diirk Watersold (Royal Guild of Baruk-Azhir),False,Noble,2,NN,0,1,0,1,2,3,4,5,5,0,0,1,Normal,[]
6,TBM,Three Brother Mages,False,Mage,9,NN,-1,2,0,3,1,0,3,0,0,0,0,1,Normal,[]
7,EL,Elamien Lamier (Points East Trading Guild),False,Noble,2,NN,0,1,0,1,2,3,4,5,5,0,0,1,Normal,[]
8,JR,Jaison Raenech,False,Fighter,7,LE,0,1,0,1,2,3,4,5,5,0,0,3,Aggresive,[Terence Gryphon]
9,WM,Willaim Moergen,False,Fighter,5,CG,0,1,0,1,2,3,4,5,5,0,0,3,Normal,[]


In [5]:
Relationships

,Regent,Other,Diplomacy,Payment,Vassalage
0,LPA,EC,3,0,1
1,JR,WM,-2,0,0
2,WM,JR,-3,0,0
